In [1]:
%load_ext autoreload
%autoreload 2

import pygsti
from pygsti.modelpacks import smq1Q_XYI
import numpy as np
from os import path as os_path
from pprint import pprint
import pandas as pd


def make_depolarized_dataset(modelpack, depol_level=0.01, max_max_len=128):
    ideal_model = modelpack.target_model()
    prep_fids = modelpack.prep_fiducials()
    meas_fids = modelpack.meas_fiducials()
    germs = modelpack.germs()
    max_lens = [2**p for p in range(1+int(np.log2(max_max_len)))]
    lsgst_circuit_lists = pygsti.circuits.create_lsgst_circuit_lists(ideal_model, prep_fids, meas_fids, germs, max_lens)
    all_circuits = lsgst_circuit_lists[-1]
    shots_per_circuit = 1000
    rng_state = np.random.default_rng(0)
    depol_model = ideal_model.depolarize(op_noise=depol_level)
    ds = pygsti.data.simulate_data(depol_model, all_circuits, shots_per_circuit, rand_state=rng_state)
    return ds


def corrupt_dataset(ds, prop_corrupt, rng=0):
    dsc = ds.copy_nonstatic()
    rng = np.random.default_rng(rng)
    num_circs = len(dsc)
    selected = rng.choice(np.arange(num_circs), size=int(num_circs*prop_corrupt), replace=False)
    circuits = list(dsc.keys())
    selected = [circuits[i] for i in selected]
    for c in selected:
        num_shots = dsc[c].total
        old_row   = dsc[c].to_dict()
        distn = rng.random(len(old_row))
        distn /= np.sum(distn)
        new_row = {k: num_shots * distn[i] for i,k in enumerate(old_row.keys())}
        dsc[c] = new_row
    dsc.comment  = 'corrupt'
    return dsc, selected


def cptp_gst(ds, fids, germs, target_model, final_objective: str, verbosity: int, mode='CPTPLND'):
    """
    In the context of this notebook, `ds` is produced by either make_depolarized_dataset or corrupt_dataset.
    final_objective can be 'tvd', 'chi2', or 'logl'.

    This function wraps up three steps of a GST pipeline.

    1. Construct a StandardGSTDesign based on (target_model, ds, fids, germs).
         * processor_spec is the value returned from target_model.create_processor_spec.
         * max_lens list is all powers of two that are <= the depth of the longest circuit in ds.
         * circuits in the design are filtered to only include circuits that appeared in ds.

    2. Construct a StandardGST protocol object based on (final_objective, mode, verbosity).
         * The gauge optimization suite is 'stdgaugeopt', minus the TPSpam optimization step.
         * objfn_builders, optimizer, and badfit_options are all set so the final 
           iteration's objective function is based on final_objective.

    3. Run GST with checkpointing turned off. 
        We dot NOT save the results to disk! The calling function is responsible for that.
    """
    max_exp = int(np.log2(np.max([len(c) for c in ds.keys()])))
    max_lens = [2**p for p in range(1 + max_exp)]
    prep_fids, meas_fids = fids

    target_model = target_model.copy()
    target_model.default_gauge_group = 'unitary'

    gos = pygsti.protocols.gst.GSTGaugeOptSuite.cast('stdgaugeopt')
    gop_params = gos.to_dictionary(target_model)
    # ^ a dict with one key, 'stdgaugeopt', whose corresponding value is a list of dicts.
    #   The internal dicts will indicate Frobenius-based losses for gates and SPAM,
    #   along with varying weights. Additional elements can be added to any one of these
    #   internal dicts to be passed to gaugeopt_to_target.
    gop_params['stdgaugeopt'] = gop_params['stdgaugeopt'][:-1]
    # ^ drop the 1-dimensional TPSpam gauge optimization step.

    exp_design = pygsti.protocols.StandardGSTDesign(
        target_model.create_processor_spec(),
        prep_fids, meas_fids, germs, max_lens,
        None,           # germ_length_limits
        None, 1, None,  # fidPairs, keepFraction, keepSeed
        True, True,     # include_lgst, nested_circuit_lists
        None,           # string_manipulation_rules
        None,           # op_label_aliases
        ds, 'drop', verbosity=verbosity
    )
    data = pygsti.protocols.ProtocolData(exp_design, ds)
    
    from pygsti.drivers.longsequence import _get_gst_builders, _get_optimizer, _get_badfit_options
    advanced_options = {'objective': final_objective}
    proto = pygsti.protocols.StandardGST(
        (mode,), gop_params, target_model, None, 
        objfn_builders    = _get_gst_builders(advanced_options),
        optimizer         = _get_optimizer(advanced_options, target_model),
        badfit_options    = _get_badfit_options(advanced_options),
        verbosity         = verbosity
    )
    results = proto.run(data, disable_checkpointing=True)
    return results



In [2]:
mp = smq1Q_XYI
target = mp.target_model()
fids = (mp.prep_fiducials(), mp.meas_fiducials())
germs = mp.germs()
ds = make_depolarized_dataset(mp, depol_level=0.01, max_max_len=128)

In [3]:
fit_mode = 'CPTPLND'

results = cptp_gst(ds, fids, germs, target, 'logl', verbosity=1, mode=fit_mode)
results.rename_estimate(fit_mode, 'fit-true-logl')
tvd_res = cptp_gst(ds, fids, germs, target, 'tvd', verbosity=1, mode=fit_mode)
results.add_estimate(tvd_res.estimates.pop(fit_mode), 'fit-true-tvd')

dsc, selected = corrupt_dataset(ds, prop_corrupt=0.025)
results_dsc = cptp_gst(dsc, fids, germs, target, 'logl', verbosity=1, mode=fit_mode)
results_dsc.rename_estimate(fit_mode, 'fit-corrupt-logl')
tvd_res  = cptp_gst(dsc, fids, germs, target,  'tvd', verbosity=1, mode=fit_mode)
results_dsc.add_estimate(tvd_res.estimates.pop(fit_mode), 'fit-corrupt-tvd')

results_dsc.add_estimate(results.estimates['fit-true-logl'], 'fit-true-logl')
results_dsc.add_estimate(results.estimates['fit-true-tvd'],  'fit-true-tvd')
results.add_estimate(results_dsc.estimates['fit-corrupt-logl'], 'fit-corrupt-logl')
results.add_estimate(results_dsc.estimates['fit-corrupt-tvd'],  'fit-corrupt-tvd')


--- Circuit Creation ---


-- Std Practice:  [##################################################] 100.0%  (CPTPLND) --
--- Circuit Creation ---


-- Std Practice:  [##################################################] 100.0%  (CPTPLND) --
--- Circuit Creation ---
-- Std Practice:  [##################################################] 100.0%  (CPTPLND) --
--- Circuit Creation ---
-- Std Practice:  [##################################################] 100.0%  (CPTPLND) --


In [4]:
report = pygsti.report.construct_standard_report(
    {'eval-true'  : results,
     'eval-corrupt' : results_dsc
    },
    advanced_options={'skip_sections': ('colorbox',)},
    title="Total variation distance (TVD) GST", verbosity=0
)

report_dir = 'example_files' if os_path.exists('example_files') else 'jupyter_notebooks/Examples/example_files'
report_dir += '/robust-gst-report'
report.write_html(report_dir, auto_open=True, verbosity=0)


/Users/rjmurr/Documents/pygsti-tvd/repo/pygsti/tools/optools.py:204: UserWarning:


            Input matrix is not PSD up to tolerance 1.8189894035458565e-12.
            The negative eigenvalues are [-0.08172067 -0.00597216].
            We'll project out the bad eigenspaces to only work with the PSD part.
            

/Users/rjmurr/Documents/pygsti-tvd/repo/pygsti/tools/optools.py:204: UserWarning:


            Input matrix is not PSD up to tolerance 1.8189894035458565e-12.
            The negative eigenvalues are [-5.02262250e-03 -7.66369607e-17].
            We'll project out the bad eigenspaces to only work with the PSD part.
            

/Users/rjmurr/Documents/pygsti-tvd/repo/pygsti/tools/optools.py:204: UserWarning:


            Input matrix is not PSD up to tolerance 1.8189894035458565e-12.
            The negative eigenvalues are [-0.0706904  -0.00501363].
            We'll project out the bad eigenspaces to only work with the PSD part.
            

/Users/rjmurr/Docume

Statistical hypothesis tests did NOT find inconsistency between the data at 5.00% significance.
The data are INCONSISTENT at 5.00% significance.
  - Details:
    - The aggregate log-likelihood ratio test is significant at 167.04 standard deviations.
    - The aggregate log-likelihood ratio test standard deviations signficance threshold is 2.00
    - The number of sequences with data that is inconsistent is 21
    - The maximum SSTVD over all sequences is 0.78
    - The maximum SSTVD was observed for Qubit 0 ---|Gxpi2|-|Gxpi2|-|Gypi2|-|Gypi2|-|Gypi2|-|Gypi2|---

The data are INCONSISTENT at 5.00% significance.
  - Details:
    - The aggregate log-likelihood ratio test is significant at 167.04 standard deviations.
    - The aggregate log-likelihood ratio test standard deviations signficance threshold is 2.00
    - The number of sequences with data that is inconsistent is 21
    - The maximum SSTVD over all sequences is 0.78
    - The maximum SSTVD was observed for Qubit 0 ---|Gxpi2|-|Gxp

In [5]:
f_dsc = results_dsc.estimates['fit-corrupt-logl'].final_objective_fn()
f_ds  = results.estimates['fit-true-logl'].final_objective_fn()

In [6]:
vals_dsc = [(en, f_dsc.fn_from_model(e.models['final iteration estimate'])) for en,e in results_dsc.estimates.items()  ]
pprint(vals_dsc)
vals_ds  = [(en, f_ds.fn_from_model(e.models['final iteration estimate']))  for en,e in results_dsc.estimates.items()  ]
pprint(vals_ds)

[('fit-corrupt-logl', 11383.347620102362),
 ('fit-corrupt-tvd', 13043.86469684735),
 ('fit-true-logl', 12987.329175096007),
 ('fit-true-tvd', 13043.840869117641)]
[('fit-corrupt-logl', 1657.4497231380374),
 ('fit-corrupt-tvd', 559.4530429064745),
 ('fit-true-logl', 549.4076481490305),
 ('fit-true-tvd', 558.3107049597204)]


In [ ]:
def sub_table_str(_df, _dsname):
    t = _df.to_markdown(tablefmt="github")
    lw = len(t.split('\n')[0])
    separator = f'|{ (lw-2)*"-" }|\n'
    caption = f"when M = argmin(F, {_dsname})"
    formatted_caption = '|' + caption.center(lw-2) + '|\n'
    s = formatted_caption + separator + t + '\n'
    return s
    

In [95]:
df  = pd.DataFrame([[v for _,v in vals_dsc if 'true'     in _], [v for _,v in vals_ds if 'true'     in _]], columns=[n for n,_ in vals_dsc if 'true'     in n], index=['corrupted','original'])
df = df.rename(columns={'fit-true-logl': 'F = -logl', 'fit-true-tvd': 'F = tvd'})
df.index.name = 'test dataset'
dfc = pd.DataFrame([[v for _,v in vals_dsc if 'true' not in _], [v for _,v in vals_ds if 'true' not in _]], columns=[n for n,_ in vals_dsc if 'true' not in n], index=['corrupted','original'])
dfc = dfc.rename(columns={'fit-corrupt-logl': 'F = -logl', 'fit-corrupt-tvd': 'F = tvd'})
dfc.index.name = 'test dataset'

def sub_table_str(_df, _dsname):
    t = _df.to_markdown(tablefmt="github")
    lw = len(t.split('\n')[0])
    separator = f'|{ (lw-2)*"-" }|\n'
    caption = f"when M = argmin(F, {_dsname})"
    formatted_caption = '|' + caption.center(lw-2) + '|\n'
    s = formatted_caption + separator + t + '\n'
    return s


In [97]:
supercaption = """\n Objective values F(M,test) on test datasets.\n"""
print(supercaption)

print(sub_table_str(df, 'original'))
print(sub_table_str(dfc, 'corrupted'))


 Objective values F(M,test) on test datasets.

|       when M = argmin(F, original)       |
|------------------------------------------|
| test dataset   |   F = -logl |   F = tvd |
|----------------|-------------|-----------|
| corrupted      |   12987.3   | 13043.8   |
| original       |     549.408 |   558.311 |

|      when M = argmin(F, corrupted)       |
|------------------------------------------|
| test dataset   |   F = -logl |   F = tvd |
|----------------|-------------|-----------|
| corrupted      |    11383.3  | 13043.9   |
| original       |     1657.45 |   559.453 |

